In [178]:
import os, time, torch, random
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import torch.nn as nn
import ipywidgets as widgets

In [1]:
trained_model_folder = "./out"
language_pair_staging_folder = "../data/magic_token_folder/"

In [8]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

In [142]:
def sample_token_numbers( model_name, token = "[GEN 1:1_a]" ):
    model_path = os.path.join( trained_model_folder, model_name )
    tokenizer = load_tokenizer(model_path)
    model = load_model(model_path)
    token_ids = tokenizer.encode( token )
    assert (len(token_ids) == 1)
    token_id = token_ids[0]
    input_embeddings = model.get_input_embeddings()
    token_embedding = input_embeddings(torch.LongTensor([token_id])).detach().numpy()[0]
    return token_embedding[:4]

Iterate through all the _step models and add up their embeddings for the magic tokens.

In [104]:
if os.path.exists(trained_model_folder) and os.path.isdir(trained_model_folder):
    # Get a list of all subfolders in language_pair_staging_folder
    subfolders = [folder for folder in os.listdir(trained_model_folder) if os.path.isdir(os.path.join(trained_model_folder, folder))]

    # Filter subfolders that end with "_model"
    model_folders = [folder for folder in subfolders if folder.endswith( "_step" ) ]

    # Print or use the list of model folders
    print("Folders ending with '_step':", model_folders)
else:
    print(f"The folder '{trained_model_folder}' does not exist or is not a directory.")

Folders ending with '_step': ['bsb_model_step', 'greek_model_step', 'target_model_step']


In [85]:
embeddings_sum = {}
embeddings_count = {}

out = widgets.Output(layout={'border': '1px solid black'})
display(out)

for model_name in model_folders:
    print( f"Grabbing embeddings from {model_name}" )
    model_path = os.path.join( trained_model_folder, model_name )
    tokenizer = load_tokenizer(model_path)
    input_embeddings = load_model(model_path).get_input_embeddings()
    target_model = model_name.replace( "_model", "" ).replace( "_step", "" )
    train_file_path = os.path.join( language_pair_staging_folder, f"train_{target_model}.txt" )
    with open( train_file_path, "rt" ) as fin:
        for line in fin:
            line_tokenized = tokenizer.encode(line)
            #print( f"We got this for the line tokenized {line_tokenized}" )
            #now decode the token and see if it is one of our tokens if it starts with [
            for token_id in line_tokenized:
                token = tokenizer.decode([token_id])
                is_magic_token = token.startswith( "[" ) and token.endswith( "]" ) and token[-3] == "_"
                if( is_magic_token ):
                    if( random.random() > .97 ):
                        out.clear_output()
                        with out:
                            print( f"Getting embedding for {token}" )
                    #print( f"I see magic token {token}" )
                    #pull out the embedding for this specific token.
                    embedding = input_embeddings(torch.LongTensor([token_id])).detach().numpy()
                    #stash away the value so we can average it out.
                    if token not in embeddings_count:
                        embeddings_count[token] = 1
                        embeddings_sum[token] = embedding
                    else:
                        embeddings_count[token] += 1
                        embeddings_sum[token] += embedding
                        

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Grabbing embeddings from ./out/bsb_model_step
Grabbing embeddings from ./out/bsb_model_step


In [198]:
#compute the actual average of the embeddings.
embeddings_average = {}
for token,embedding_sum in embeddings_sum.items():
    embeddings_average[token] = embedding_sum/embeddings_count[token]

In [101]:
#print out a sample of the embeddings before updating them so we can see if they changed and how.
subfolders = [folder for folder in os.listdir(trained_model_folder) if os.path.isdir(os.path.join(trained_model_folder, folder))]
all_model_folders = [folder for folder in subfolders if folder.endswith( "_model" ) or folder.endswith( "_step" ) ]
for model_name in all_model_folders:
    print( f"{model_name}: {sample_token_numbers( model_name )}" )

macula_model: [ 0.01821761 -0.03325709 -0.01717107 -0.01796533]
bsb_model_step: [ 0.01380756 -0.06270172  0.05579207 -0.01744836]
greek_model: [ 0.01821761 -0.03325709 -0.01717107 -0.01796533]
greek_model_step: [ 0.03959837 -0.01270676  0.065974   -0.05270576]
target_model_step: [-0.04399784 -0.05841777  0.05558705  0.03709033]
bsb_model: [ 0.01821761 -0.03325709 -0.01717107 -0.01796533]
target_model: [ 0.01821761 -0.03325709 -0.01717107 -0.01796533]
hebrew_model: [ 0.01821761 -0.03325709 -0.01717107 -0.01796533]


In [106]:
model_folders

['bsb_model_step', 'greek_model_step', 'target_model_step']

In [199]:
#now run back through all the models and inject the average embeddings.
for model_name in model_folders:
    #load the tokenizer and model
    model_path = os.path.join( trained_model_folder, model_name )
    print( f"Pulling in {model_path}" )
    tokenizer = load_tokenizer(model_path)
    model = load_model(model_path)
    input_embeddings = model.get_input_embeddings()
    
    reconstructed_embedding_list = []
    #run through all the tokens
    for token_id in range(len(tokenizer)):
        token = tokenizer.decode([token_id])
        #and grab them from the average if it is in there.
        if token in embeddings_average:
            reconstructed_embedding_list.append( embeddings_average[token][0] )
        else:
            reconstructed_embedding_list.append( input_embeddings(torch.LongTensor([token_id])).detach().numpy()[0] )
            
    #construct it into some weights.
    new_weights = torch.FloatTensor(reconstructed_embedding_list)
    updated_embedding = nn.Embedding.from_pretrained(new_weights)
    #put it back into the model
    model.set_input_embeddings( updated_embedding )
    model.tie_weights()

    #save the model back to the non _step locations so the loop can begin again.    
    output_dir = model_path.replace( "_step", "" )
    print( f"Updating {output_dir}" )
    model.save_pretrained(output_dir)

Pulling in ./out/bsb_model_step
Updating ./out/bsb_model
Pulling in ./out/greek_model_step
Updating ./out/greek_model
Pulling in ./out/target_model_step
Updating ./out/target_model


In [201]:
#now run through all the models and sample their numbers again to see how they changed.
subfolders = [folder for folder in os.listdir(trained_model_folder) if os.path.isdir(os.path.join(trained_model_folder, folder))]
all_model_folders = [folder for folder in subfolders if folder.endswith( "_model" ) or folder.endswith( "_step" ) ]
for model_name in all_model_folders:
    print( f"{model_name}: {sample_token_numbers( model_name )}" )

bsb_model_step: [ 0.01380756 -0.06270172  0.05579207 -0.01744836]
greek_model: [ 0.01380756 -0.06270172  0.05579207 -0.01744836]
greek_model_step: [ 0.03959837 -0.01270676  0.065974   -0.05270576]
target_model_step: [-0.04399784 -0.05841777  0.05558705  0.03709033]
bsb_model: [ 0.01380756 -0.06270172  0.05579207 -0.01744836]
target_model: [ 0.01380756 -0.06270172  0.05579207 -0.01744836]
hebrew_model: [ 0.01821761 -0.03325709 -0.01717107 -0.01796533]
